# TODO

explain NUTS to SSB


In [ ]:
nuts_to_ssb = {
    "NO081": "NO03",
    "NO082": "NO30",
    "NO020": "NO34",
    "NO091": "NO38",
    "NO092": "NO42",
    "NO0A1": "NO11",
    "NO0A2": "NO46",
    "NO0A3": "NO15",
    "NO060": "NO50",
    "NO071": "NO18",
    "NO074": "NO54",
}

In [ ]:
sharedinputpath = snakemake.params["sharedinputpath"]

# TODO

create the 2013_temporal.dd file


In [ ]:
# pd.Series(range(0,8760)).div(10000)

# TODO

explain input files


In [ ]:
# Describe what they mean.
onshorepath = sharedinputpath / "geodata/onshore/"
WDPA_Ia = onshorepath / "WDPA_Ia_100.tiff"
WDPA_Ib = onshorepath / "WDPA_Ib_100.tiff"
WDPA_II = onshorepath / "WDPA_II_100.tiff"
WDPA_III = onshorepath / "WDPA_III_100.tiff"
WDPA_IV = onshorepath / "WDPA_IV_100.tiff"

# TODO

explain choice for on and offshore turbine


In [ ]:
onshore_turbine = "Vestas_V112_3MW"
offshore_bottom_turbine = "oedb:V164"
offshore_floating_turbine = "oedb:V164"

# TODO

explain heightshp


In [ ]:
# >2000m ausgeschlossen
heightshp = onshorepath / "2000m.shp.zip"

# TODO

explain degreeshape


In [ ]:
# >15 deg hangneigung ausgeschlossen
degreeshp = onshorepath / "15degrees.shp.zip"

# TODO

explain the choice of the input desired regions


In [ ]:
# Svalbard and Jan Mayen 27 EU + UK + Switzerland+ Norway (fylke)   
desired_regions = [
    "NO020",
    "NO060",
    "NO071",
    "NO074",
    "NO081",
    "NO082",
    "NO091",
    "NO092",
    "NO0A1",
    "NO0A2",
    "NO0A3",
]

# TODO

explain choice of codes


In [ ]:
# solar codes choice based on what sounded good
#codes = (
#    2,
#    4,
#    5,
#    6,
#    10,
#    11,
#    12,
#    13,
#    14,
#    15,
#    16,
#    17,
#    18,
#    19,
#    20,
#    21,
#    22,
#    23,
#    24,
#    25,
#    34,
#    35,
#    36,
#    37,
#    38,
#    39,
#    40,
#    41,
#    42,
#    43,
#    44,
#)

In [ ]:
# Corine solar codes for that actually need to be included [invert = true]
corine_solar_codes = snakemake.params.corine_solar

In [ ]:
corine_solar_codes

In [ ]:
# explain what the codes mean
#wind_onshore_codes_no_buffer = (
#    3,
#    4,
#    5,
#    7,
#    8,
#    9,
#    10,
#    11,
#    34,
#    35,
#    36,
#    37,
#    38,
#    39,
#    40,
#    41,
#)

# find reasoning for why buffers differ
wind_onshore_codes_buffer = {1: 2000, 2: 1000, 6: 5000}
#wind_onshore_codes_buffer = {1: 2000, 6: 5000}

In [ ]:
corine_onshore_codes = snakemake.params.corine_onshorewind

In [ ]:
date_range=snakemake.params.date_range
date_range

In [ ]:
year = snakemake.wildcards.year

In [ ]:
date_range=[year+"-"+date for date in date_range]
date_range

# TODO

explain the choice of 2013

2013 "industry standard"
1987 aleks difficult year


# TODO

explain the choice of offshore files

exclusive economic area is the outline and depth (should be specified somewhere)


# TODO

explain the choice of rectangle borders


In [ ]:
# The year for which to calculate the weather data:
#year = snakemake.wildcards.year                                                                         #this year is exlained where?

# The country for which to calculate the weather data:
desired_country = "NO"

# Geodata files to use for selecting country onshore and offshore area:
geodata_files = {
    "onshore": snakemake.input["euroshape"],
    "offshore_bottom": snakemake.input["eurooffshoreshape"],
    "offshore_floating" : snakemake.input["eurooffshorefloatingshape"],
    # "offshore_future" : "offshore_geodata/FUTURE_EUROPE_NUTS0_NORWAY_NUTS3.geojson"
}

# The square outer boundaries of Europe to consider, because we have downloaded ERA5 for this extent:
rectx1 = -12
rectx2 = 44
recty1 = 33
recty2 = 81

- bottom 0 - 70 m depth
- floating 70 - 1000 m depth
- future 1000 - m depth # ignore


# TODO

explain where the 2013.nc file comes from


In [ ]:
weatherdata = sharedinputpath / ("weatherdata/europe_" + str(year) + ".nc")

# TODO

Explain where corine came from


In [ ]:
CORINE = sharedinputpath / "geodata/onshore/corine.tif"

# TODO

Think about excluding low capacity factor areas. Currently disabled.


# TODO

epxlain choice of panel and orientation


In [ ]:
panel = "CSi"
orientation = "latitude_optimal"

# TODO

create the file geojson file with Norway NUTS3 and rest NUTS0


In [ ]:
file_name = geodata_files["onshore"]
excelurl = "https://uio-my.sharepoint.com/:u:/g/personal/maximilr_uio_no/EYYGvCuWhlFFvjLe0JKlz1ABwq1eBHRa72zbivOJtK6pQQ?e=LlFIDB&download=1"

We need to replace the regions by grid cell ids.

1. the file dev_regions needs to contain the names of the regions

2. (optional) currently, hydro res only runs on zonal level, here we could include regions as well

3. vre_2013_dev.gdx needs to be created for all regions

4. vre_areas_2013_dev.dd needs to be created for all regions

exist_vre_cap_r
vre_lim
area
var_new_vre_pcap_r
var_exist_vre_pcap_r
var_vre_gen_r
var_vre_curtail
var_vre_pcap_r
vre_gen
var_new_vre_pcap_r
var_vre_curtail


In [ ]:
import logging

import atlite

logging.basicConfig(level=logging.INFO)

import io
import os
import pathlib

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import xarray as xr
from atlite.gis import ExclusionContainer, shape_availability
from shapely.geometry import Polygon

# from pyhighRES import constants

In [ ]:
atlite.__version__  # should be 0.2.4

In [ ]:
xr.__version__  # should be 0.18.2

In [ ]:
# should include the extra maybe progressbar code without time

In [ ]:
%psource atlite.Cutout.convert_and_aggregate

# Desired regions


In [ ]:
def download(url, file_name):
    if not os.path.exists(file_name):
        # open in binary mode
        with open(file_name, "wb") as file:
            # get request
            response = requests.get(url)
            # write to file
            file.write(response.content)

In [ ]:
desired_regions

## Shape level

To get the desired data at shape level, the first input is the shapefile itself.
capacity_factor_raw = da ohne mean in convert and aggregate


In [ ]:
# NUTS_RG_60M_2021_4326_NO_LEVL_3_REST_LEVL_0.geojson
# url = "https://uio-my.sharepoint.com/:u:/g/personal/maximilr_uio_no/EYYGvCuWhlFFvjLe0JKlz1ABwq1eBHRa72zbivOJtK6pQQ?e=LlFIDB&download=1"
# urlData = requests.get(url).content
# spurl = io.StringIO(urlData.decode('utf-8'))
# europe = gpd.read_file(spurl).replace({"UK":"GB","EL":"GR"}).rename(columns={"id":"index"}).loc[:,["index","geometry"]].sort_values("index").set_index('index').drop(["AL","ME","MK","NO0B1","NO0B2","RS","TR","LI"])
# europe = gpd.read_file("NUTS_RG_10M_2021_4326_europe_LEVL_0.geojson").set_index('index')

In [ ]:
file_name

In [ ]:
excelurl

In [ ]:
# download(excelurl,file_name)
europe = (
    gpd.read_file(file_name)
    .replace({"UK": "GB", "EL": "GR"})
    .rename(columns={"NUTS_ID": "index"})
    .loc[:, ["index", "geometry"]]
    .sort_values("index")
    .set_index("index")
    .loc[desired_regions]
)
# europe = gpd.read_file("NUTS_RG_10M_2021_4326_europe_LEVL_0.geojson").set_index('index')

In [ ]:
europe = europe[europe.index.str.contains("NO")]

In [ ]:
europe

In [ ]:
year

In [ ]:
rectx1

In [ ]:
rectx2

In [ ]:
recty1

In [ ]:
recty2

In [ ]:
polygon = Polygon(
    [
        (rectx1, recty1),
        (rectx1, recty2),
        (rectx2, recty2),
        (rectx2, recty1),
        (rectx1, recty1),
    ]
)
europe = gpd.clip(europe, polygon)

In [ ]:
# europe = europe.drop(index = 'IS') #removes Iceland as it is not included in the ERA5 cutout
# europe = europe.loc[["AT","DE"]]
europe.plot(figsize=(15, 15))

# Cutout preparation


In [ ]:
gpd.read_file(
    sharedinputpath
    / "geodata/offshore/BOTTOM_MOUNTED_EUROPE_NUTS0_NORWAY_NUTS3.geojson"
)

In [ ]:
boundaries = []
for geodata_file_name, geodata_file_path in geodata_files.items():
    print(geodata_file_path)
    boundaries.append(
        gpd.read_file(geodata_file_path)
        .rename(columns={"index": "old_index"})
        .rename(columns={"NUTS_ID": "index"})
        .set_index("index")
        .rename({"United Kingdom": "UK", "GB": "UK"})
        # Since we do not need Svalbard and Jan Mayen, we drop them here. Since not all GeoDataFrames we are looping through
        # might have them, we ignore errors.
        .drop(["NO0B1", "NO0B2"], errors="ignore")
        # Here we filter to only include Norway.
        .filter(like=desired_country, axis=0)
        # .cascaded_union
        # .buffer(1)
        .bounds
    )

boundaries = pd.concat(boundaries)

In [ ]:
boundaries

In [ ]:
boundaries = boundaries.groupby(lambda x: desired_country).agg(
    {"minx": "min", "miny": "min", "maxx": "max", "maxy": "max"}
)

In [ ]:
boundaries["minx"] = boundaries["minx"] - 2
boundaries["miny"] = boundaries["miny"] - 2
boundaries["maxx"] = boundaries["maxx"] + 2
boundaries["maxy"] = boundaries["maxy"] + 2
# boundaries["minx"] = 0.0
# boundaries["miny"] = 55.0
# boundaries["maxx"] = 5.0
# boundaries["maxy"] = 72.0
boundaries

In [ ]:
weatherdata

In [ ]:
cutout = atlite.Cutout(path=weatherdata)

In [ ]:
cutout.prepared_features

In [ ]:
cutout.prepare()

In [ ]:
cutout = atlite.Cutout(
    path="../3_intermediate_data/intermediatecutout.nc",            #generating cutout consdering given boundries, including offshore
    # module = "era5",
    # time = "2014",
    # bounds = europe.cascaded_union.buffer(1).bounds,
    data=cutout.data.sel(
        x=slice(
            boundaries.loc[desired_country, "minx"],
            boundaries.loc[desired_country, "maxx"],
        ),
        y=slice(
            boundaries.loc[desired_country, "miny"],
            boundaries.loc[desired_country, "maxy"],
        ),
        # time = slice('2010-01-01','2010-01-07')
    ),
)

In [ ]:
cutout.prepare()

In [ ]:
cutout.data

In [ ]:
# limiting the simulation days of the year
cutout.data = cutout.data.sel(time=slice(date_range[0], date_range[1]))

In [ ]:
cutout.data

## Indices for gridcells


In [ ]:
gridcellnamingfunction = (
    lambda x: "x"
    + (x.x * 100).astype("int").astype("str")
    + "y"
    + (x.y * 100).astype("int").astype("str")
)

In [ ]:
if snakemake.wildcards.spatial == "grid":
    (
        cutout.grid.assign(gridcell=gridcellnamingfunction)
        .loc[:, ["gridcell"]]
        .to_csv(snakemake.output["indreg"], header=False, index=False)
    )
else:
    with open(snakemake.output["indreg"], "w"):
        pass

# Solar


## Areas


In [ ]:
panel

In [ ]:
orientation

In [ ]:
excluder_solar = ExclusionContainer()

In [ ]:
CORINE

In [ ]:
#codes
corine_solar_codes

In [ ]:
#excluder_solar.add_raster(CORINE, codes=codes)
excluder_solar.add_raster(CORINE, codes=corine_solar_codes, invert=True)

In [ ]:
WDPA_Ia

In [ ]:
WDPA_Ib

In [ ]:
WDPA_II

In [ ]:
WDPA_III

In [ ]:
WDPA_IV

In [ ]:
excluder_solar.add_raster(WDPA_Ia)
excluder_solar.add_raster(WDPA_Ib)
excluder_solar.add_raster(WDPA_II)
excluder_solar.add_raster(WDPA_III)
excluder_solar.add_raster(WDPA_IV)

In [ ]:
degreeshp

In [ ]:
slope = gpd.read_file(degreeshp).to_crs(excluder_solar.crs)

slope = slope[slope["gridcode"] == 1]

# use invert=True to exclude the areas that are to steep instead of excluding everything else
excluder_solar.add_geometry(slope.geometry, invert=True)

In [ ]:
heightshp

In [ ]:
height = gpd.read_file(heightshp).to_crs(excluder_solar.crs)

height = height[height["gridcode"] == 1]

excluder_solar.add_geometry(height.geometry, invert=True)

In [ ]:
# cf = cutout.pv(panel=panel, orientation=orientation, capacity_factor=True)

In [ ]:
# smallestincluded = 0.11

In [ ]:
# excluded = cf.where(cf.values>=smallestincluded, other=0)
# excluded = excluded.where(excluded.values<smallestincluded, other=1)

In [ ]:
# cf_exclusion = excluded.rio.write_crs(europe.crs)

In [ ]:
# cf_exclusion.rio.to_raster('data/cf_exclusion.tif')

In [ ]:
#@gridcell level: model will strategically exclude the gridcells having low CF
if snakemake.wildcards.spatial == "grid":
    for exclusion_tech_type in [
        "cf_exclusion_solar",
        "cf_exclusion_windon",
        "cf_exclusion_windoff",
    ]:
        with open(snakemake.output[exclusion_tech_type], "w"):
            pass

In [ ]:
# exclusion based on the capacity factors & code based exclusion is done with CORINE 
def cutoff_raster(technology,output):
    if technology == 'pv':
        cf = cutout.pv(panel=panel, orientation=orientation, capacity_factor=True)
        #smallestincluded = snakemake.params.cutoffs["solar"]
        smallestincluded = float(snakemake.wildcards.cutoff_solar)

    if technology == 'onwind':
        cf = cutout.wind(turbine=onshore_turbine, capacity_factor=True)
        #smallestincluded = snakemake.params.cutoffs["onwind"]
        smallestincluded = float(snakemake.wildcards.cutoff_onwind)

    if technology == 'offwind':
        cf = cutout.wind(turbine=offshore_bottom_turbine, capacity_factor=True)
        #smallestincluded = snakemake.params.cutoffs["offwind"]
        smallestincluded = float(snakemake.wildcards.cutoffs_offwind)

    if technology == "offwind_floating":
        cf = cutout.wind(turbine=offshore_floating_turbine, capacity_factor=True)
        smallestincluded = float(snakemake.wildcards.cutoffs_offwind)
        
    excluded = cf.where(cf.values>=smallestincluded, other=1)
    excluded = excluded.where(cf.values<smallestincluded, other=0)
    cf_exclusion = excluded.rio.write_crs(europe.crs)
    cf_exclusion.rio.to_raster(output)

In [ ]:
# we need to define cutoffs as param of this rule
# added the cutoff_raster function
low_cf = snakemake.output.cf_exclusion_solar
#if snakemake.wildcards.spatial == "region" and snakemake.params.cutoffs["solar"] != 0:
if snakemake.wildcards.spatial == "region" and float(snakemake.wildcards.cutoff_solar) != 0:    
    cutoff_raster("pv", low_cf)
    excluder_solar.add_raster(low_cf)
# low_cf = 'data/cf_exclusion.tif'
# excluder_solar.add_raster(low_cf, invert=True)

In [ ]:
# what pvlow is excluding...?
excluder_solar.add_geometry(
    gpd.read_file(snakemake.input["pvlow"]).to_crs(excluder_solar.crs).geometry
)

In [ ]:
availability_matrix_solar = cutout.availabilitymatrix(
    europe, excluder_solar, nprocesses=snakemake.threads
)
availability_matrix_solar

In [ ]:
# what cap_per_sqkm is doing?

cap_per_sqkm = 1  # see if the results are the same when changing this

area = cutout.grid.set_index(["x", "y"]).to_crs(3035).area / 1e6

area = xr.DataArray(area, dims=("spatial"))

# weighted average area of each grid_cell based on the capacity_factor_cutoffs
capacity_matrix_solar = (
    availability_matrix_solar.stack(spatial=["x", "y"]) * area * cap_per_sqkm
)

# the following is the crucial addition
capacity_matrix_solar = capacity_matrix_solar.reindex(
    spatial=area.indexes.get("spatial")
)

capacity_matrix_solar

In [ ]:
availability_matrix_solar.stack(
    spatial=["index", "x", "y"]
).to_pandas().reset_index().assign(gridcell=gridcellnamingfunction).set_index(
    ["index", "gridcell"]
).loc[:, [0]].round(1)

In [ ]:
(
    capacity_matrix_solar.unstack()
    .stack(spatial=["index", "x", "y"])
    .to_pandas()
    .reset_index()
    .assign(gridcell=gridcellnamingfunction)
    .set_index(["index", "gridcell"])
    .loc[:, [0]]
    .round(1)
)

In [ ]:
# constants.
nuts_to_ssb

In [ ]:
if snakemake.wildcards.spatial == "grid":
    highRESareasSolar = (
        capacity_matrix_solar.unstack()
        .stack(spatial=["index", "x", "y"])
        .to_pandas()
        .reset_index()
        .replace({"index": nuts_to_ssb})
        .assign(
            gridcell=gridcellnamingfunction,
            new_idx=lambda x: "Solar." + x["index"] + "." + x["gridcell"],
        )
        .set_index("new_idx")
        .drop(columns=["index"])
        .loc[:, [0]]
        ## the following will drop rows with a value of zero, which will significantly reduce input file size.
        ## downside is that the indices need to be processed seperately then because gams needs an exhaustive list of all index values
        .rename(columns={0: "area"})
        .query("area != 0")
    )

In [ ]:
if snakemake.wildcards.spatial == "region":
    highRESareasSolar = (
        capacity_matrix_solar.sum("spatial")
        .to_pandas()
        .reset_index()
        .replace({"index": nuts_to_ssb})
        .assign(
            new_idx=lambda x: "Solar." + x["index"] + "." + x["index"],
        )
        .set_index("new_idx")
        .drop(columns=["index"])
        .loc[:, [0]]
        .rename(columns={0: "area"})
        .query("area != 0")
    )

In [ ]:
highRESareasSolar

In [ ]:
highRESareasSolar.sum()

In [ ]:
(
    highRESareasSolar.round(1).to_csv(
        snakemake.output["areassolar"], header=False, sep=" "
    )
)

## Capacity factors


Here we need to use the following parameters:

- matrix (N x S - xr.DataArray or sp.sparse.csr_matrix or None) – If given, it is used to aggregate the grid cells to buses. N is the number of buses, S the number of spatial coordinates, in the order of cutout.grid.
  - Since we want to aggregate grid cells to shapes (countries), which are called buses in atlite, this is necessary.
- per_unit (boolean) – Returns the time-series in per-unit units, instead of in MW (defaults to False).
  - Since we want capacity factors and not capacities, we have to use per_unit. Per-unit units means the factor between 0 and 1 of the total capacity. Since we are using this relative measure, the total capacity does not matter and the capacity_matrix should therefor only be used to exclude cells.
- index (pd.Index) – Index of Buses.
  - This is the index of the shapes.


In [ ]:
# Why matrix is not passed with grid condition? 

if snakemake.wildcards.spatial == "grid":
    _, capacity_factor_solar = cutout.pv(
        panel=panel,
        orientation=orientation,
        # matrix=capacity_matrix_solar,
        capacity_factor=True,  # does not do anything, if you have no matrix or layout or shapes
        # per_unit=True,
        # index=europe.index
    )
    
if snakemake.wildcards.spatial == "region":
    capacity_factor_solar = cutout.pv(
        panel=panel,
        orientation=orientation,
        matrix=capacity_matrix_solar,
        per_unit=True,
        # matrix=capacity_matrix_solar,
        capacity_factor=True,  # does not do anything, if you have no matrix or layout or shapes
        # per_unit=True,
        index=capacity_matrix_solar.index,
    )

capacity_factor_solar

In [ ]:
capacity_factor_solar

In [ ]:
# (
#    capacity_factor_solar.stack(spatial=["x", "y"])
#    .to_pandas()
#    .stack()
#    .stack()
#    .reset_index()
#    .assign(gridcell=gridcellnamingfunction, technology="Solar")
#    .set_index(["time", "technology", "gridcell"])
#    .drop(columns=["x", "y"])
#    .to_xarray()
#    # .round(
#    #    1
#    # )
#    # .loc[
#    #    :,
#    #    [0]
#    # ]
#    # )
#    # .set_index()
#    # .to_xarray()
#    # .loc[
#    #    :,
#    #    ["gridcell"]
#    # ]
#    # .to_csv(
#    #    "indices_region.csv",
#    #    header = False,
#    #    index = False
#    # )
# )  # .info()

# Wind


In [ ]:
excluder_wind_onshore = ExclusionContainer()

In [ ]:
# cutoffexclusiontiff contains only gridcells having value higher than cuttoff. it contains the yearly mean corrected CF value of each grid cell.
#if snakemake.wildcards.spatial == "region" and snakemake.wildcards.cutoff_wind != str(0.0):
if snakemake.wildcards.spatial == "region" and snakemake.wildcards.cutoff_onwind != str(0.0):
    excluder_wind_onshore.add_raster(snakemake.input["cutoffexclusiontiff"])

In [ ]:
#wind_onshore_codes_no_buffer

In [ ]:
wind_onshore_codes_buffer

In [ ]:
# adding CORINE codes for onshore landscapes exclusion
excluder_wind_onshore.add_raster(CORINE, codes=corine_onshore_codes, invert=True)

for code, buffer in wind_onshore_codes_buffer.items():
    excluder_wind_onshore.add_raster(CORINE, codes=(code), buffer=buffer)

In [ ]:
# WDPA addition
excluder_wind_onshore.add_raster(WDPA_Ia)
excluder_wind_onshore.add_raster(WDPA_Ib)
excluder_wind_onshore.add_raster(WDPA_II)
excluder_wind_onshore.add_raster(WDPA_III)
excluder_wind_onshore.add_raster(WDPA_IV)

In [ ]:
degreeshp

In [ ]:
slope = gpd.read_file(degreeshp).to_crs(excluder_wind_onshore.crs)

slope = slope[slope["gridcode"] == 1]

excluder_wind_onshore.add_geometry(
    slope.geometry, invert=True
)  # use invert=True to exclude the areas that are to steep instead of excluding everything else

In [ ]:
heightshp

In [ ]:
height = gpd.read_file(heightshp).to_crs(excluder_wind_onshore.crs)

height = height[height["gridcode"] == 1]

excluder_wind_onshore.add_geometry(height.geometry, invert=True)

In [ ]:
# these codes are different from the one defined WRT CORINE at the very start
# this is a NVE file and particulalry related NORWAY
excluder_wind_onshore.add_raster(snakemake.input["windlow"], codes=[0])

In [ ]:
# this exclusion overlaps with the one done using bias-corrected onwind CF
#if snakemake.wildcards.spatial == "region" and snakemake.params.cutoffs["onwind"] != 0:
#    low_cf = snakemake.output.cf_exclusion_windon
#    cutoff_raster("onwind", low_cf)
#    excluder_wind_onshore.add_raster(low_cf)
    
# tech: A buffer of 200m is applied to roads and railway in case a wind turbine falls or breaks
#(Enevoldsen and Permien 2018). An Airport 2km was added due to safety. Watercourses,
# water bodies and glaciers were exluded since these would require offshore wind turbines and foundations.
#excluder_wind_onshore.add_geometry(
#    gpd.read_file(snakemake.input["tech"]).to_crs(excluder_wind_onshore.crs).geometry
#)

In [ ]:
# Added neigh_Low_Exc as base exclusion in windonshore
#excluder_wind_onshore.add_geometry(
#    gpd.read_file(snakemake.input["neighex"]).to_crs(excluder_wind_onshore.crs).geometry
#)

In [ ]:
# Here we have to deal with the neighbour scenario.
# For Norway we know where the turbines are located, but we need to convert that
# to areas. We can do this by dividing the installed capacity of the farm by
# the assumed capacity density in power/area to receive the area.

# We should be able to suck that out of the excel sheet.

# Once we have the file we can add it with
# add_geometry(self, geometry, buffer=0, invert=True)

In [ ]:
#Mypoint: Contain the existing wind farm areas regardless of exclusion scenario & simulation year
# assuming that wind trubine installed can be replaced with new ones after useful lifetime 

In [ ]:
#antithesis: it is not necessarily economically just locations where currently WT are installed. Letting the
# model to install WT based on people input. 2nd point is currently installed WT would be expired by 2050 and
# it is not necessary to install the new ones on the same places yet it is true that infrastructure/transmission lines
# are there and we are using it as input of exisiting transmission lines which may kind of forcing the model to
# install WT in same counties (yes the location may change but in same county)

In [ ]:
# to convert it into object from string
#windcap2area = float(pathlib.Path(snakemake.input["cap2area"]).read_text())

In [ ]:
#windcap2area  # MW/km2 its 3.0

In [ ]:
#wind_parks = snakemake.input["wind_parks"]

In [ ]:
#windparks = gpd.read_file(wind_parks)

In [ ]:
#windparks

In [ ]:
#buffering = (
#    windparks.query("effekt_MW_idrift > 0")
#    .assign(
#        km2=lambda df: df.effekt_MW_idrift / windcap2area,
#        radius=lambda df: (np.sqrt(df.km2 / np.pi)) * 1000,  # in metres now
#    )
#    .loc[:, ["anleggNavn", "effekt_MW_idrift", "radius", "geometry"]]
#)

In [ ]:
#buffering.assign(x=lambda df: df.geometry.x, y=lambda df: df.geometry.y).rename(
#    columns={"effekt_MW_idrift": "capacity"}
#)

In [ ]:
# wind_layout = cutout.layout_from_capacity_list(buffering.assign(x= lambda df: df.geometry.x,y= lambda df: df.geometry.y).rename(columns={"effekt_MW_idrift":"capacity"}), col="capacity")

In [ ]:
# wind_layout.plot(cmap="inferno_r", size=8, aspect=1)

In [ ]:
# windactprod = cutout.wind(turbine=onshore_turbine,layout=wind_layout)

In [ ]:
# This corresponds to about 13.1 TWh in a normal year.
# windactprod.sum()

In [ ]:
#buffering.to_crs(epsg=6933).buffer(buffering.radius).to_crs(epsg=4326).plot()

In [ ]:
# buffering contains: windfarmName, capacity, radius, geometry, x, y
# Excluding all currently installed wind turbine areas 
#excluder_wind_onshore.add_geometry(
#    buffering.to_crs(epsg=6933)
#    .buffer(buffering.radius)
#    .to_crs(excluder_wind_onshore.crs)
#    .geometry,
#)

In [ ]:
# solution delete everything we do not want, plus delete existing wind, then add matrix with existing wind.

In [ ]:
availability_matrix_wind_onshore = cutout.availabilitymatrix(
    europe, excluder_wind_onshore
)
availability_matrix_wind_onshore

In [ ]:
#excluder_wind_onshore_existing = ExclusionContainer()

In [ ]:
# Here adding the existing wind trubine areaonly to calculate exisiting availability matrix
#excluder_wind_onshore_existing.add_geometry(
#    buffering.to_crs(epsg=6933)
#    .buffer(buffering.radius)
#    .to_crs(excluder_wind_onshore.crs)
#    .geometry,
#    invert=True,
#)

In [ ]:
#availability_matrix_wind_onshore_existing = cutout.availabilitymatrix(
#    europe, excluder_wind_onshore_existing
#)
#availability_matrix_wind_onshore_existing

In [ ]:
availability_matrix_wind_onshore.sum()

In [ ]:
#availability_matrix_wind_onshore_existing.sum()

In [ ]:
#availability_matrix_wind_onshore = (
#    availability_matrix_wind_onshore + availability_matrix_wind_onshore_existing
#)

In [ ]:
cap_per_sqkm = 1  # see if the results are the same when changing this

area = cutout.grid.set_index(["x", "y"]).to_crs(3035).area / 1e6

area = xr.DataArray(area, dims=("spatial"))

capacity_matrix_wind_onshore = (
    availability_matrix_wind_onshore.stack(spatial=["x", "y"]) * area * cap_per_sqkm
)

# the following is the crucial addition
capacity_matrix_wind_onshore = capacity_matrix_wind_onshore.reindex(
    spatial=area.indexes.get("spatial")
)

# capacity_matrix_wind_onshore is at gridcell level 
capacity_matrix_wind_onshore

In [ ]:
if snakemake.wildcards.spatial == "grid":
    highRESareasWindOnshore = (
        capacity_matrix_wind_onshore.unstack()
        .stack(spatial=["index", "x", "y"])
        .to_pandas()
        .reset_index()
        .replace({"index": nuts_to_ssb})
        .assign(
            gridcell=gridcellnamingfunction,
            new_idx=lambda x: "Windonshore." + x["index"] + "." + x["gridcell"],
        )
        .set_index("new_idx")
        .drop(columns=["index"])
        .loc[:, [0]]
        ## the following will drop rows with a value of zero, which will significantly reduce input file size.
        ## downside is that the indices need to be processed seperately then because gams needs an exhaustive list of all index values
        .rename(columns={0: "area"})
        .query("area != 0")
    )

if snakemake.wildcards.spatial == "region":
    highRESareasWindOnshore = (
        capacity_matrix_wind_onshore.sum("spatial")
        .to_pandas()
        .reset_index()
        .replace({"index": nuts_to_ssb})
        .assign(
            new_idx=lambda x: "Windonshore." + x["index"] + "." + x["index"],
        )
        .set_index("new_idx")
        .drop(columns=["index"])
        .loc[:, [0]]
        .rename(columns={0: "area"})
        .query("area != 0")
    )

In [ ]:
highRESareasWindOnshore

In [ ]:
# if snakemake.wildcards.neigh != "Extreme":
(
    highRESareasWindOnshore.round(1).to_csv(
        snakemake.output["areaswindonshore"], header=False, sep=" "
    )
)

# if snakemake.wildcards.neigh == "Extreme":
#    with open(snakemake.output["areaswindonshore"], 'w'):
#        pass

In [ ]:
onshore_turbine

In [ ]:
if snakemake.wildcards.spatial == "grid":
    (
        pd.read_parquet(snakemake.input.changedbiaswind)
        .unstack()
        .unstack()
        .reset_index(drop=True)
        .stack()
        .stack()
        .round(2)
        .to_csv(snakemake.output.changedbiaswindcsv, header=False)
    )

In [ ]:
if snakemake.wildcards.spatial == "region":
    cf = (
        pd.read_parquet(snakemake.input.changedbiaswind)
        .reset_index()
        .set_index("spatial")
    )

    area = (
        capacity_matrix_wind_onshore.to_pandas()
        .unstack()
        .to_frame()
        .rename(nuts_to_ssb, axis=0)
        .reset_index()
        .assign(gridcell=gridcellnamingfunction)
        .set_index(["index", "gridcell"])
        .loc[:, 0]
        .to_frame()
        .rename_axis(index={"gridcell": "spatial"})
        .reset_index()
        .set_index("spatial")
        .rename(columns={0: "cf"})
    )

    # Calculate the share of each grid cell's area on the total area in each county
    shares = (
        area.set_index(["index"], append=True)
        .div(
            # Calculate the total available area in each county
            area.groupby("index").sum()
        )
        .reset_index()
        .set_index("spatial")
        .rename(columns={"cf": "area"})
    )

    # Match capacity factors and share of each grid cell's area
    merged = (
        pd.merge(shares, cf, on="spatial")
        .reset_index()
        .set_index(["index", "spatial", "time", "technology"])
        .sort_index()
        .rename(columns={"0": "cf"})
    )

    # Add a column multiplying the capacity factors with the grid cell's area share
    merged["areaXcf"] = merged["area"] * merged["cf"]

    # Sum all grid cells in the same region
    capacity_factor_wind_onshore = (
        merged.groupby(["time", "technology", "index"]).sum().loc[:, ["areaXcf"]]
    )

    (
        capacity_factor_wind_onshore.unstack()
        .unstack()
        .reset_index(drop=True)
        .stack()
        .stack()
        .round(2)
        .to_csv(snakemake.output.changedbiaswindcsv, header=False)
    )

In [ ]:
# Method chained version of the above

# cf_weighted_mean_region = (
#    pd.read_csv('areas.csv').set_index(['spatial'])
#    .merge(pd.read_csv('bias_corrected_capacity_factors.csv').set_index(['spatial']), on='spatial')
#    .reset_index()
#    .set_index(['spatial', 'index', 'time', 'technology'])
#    .rename(columns={'0_x':'area', '0_y':'cf'})
#    .assign(areaXcf= lambda df: df.area * df.cf)
#    .groupby(['index','time','technology']).sum()
#    .assign(weighted_mean= lambda df: df.areaXcf / df.area)
#    .dropna()
# )

In [ ]:
# _, capacity_factor_wind_onshore = cutout.wind(
#    turbine=onshore_turbine, capacity_factor=True
# )
# capacity_factor_wind_onshore

In [ ]:
# (
#    capacity_factor_wind_onshore.stack(spatial=["x", "y"])
#    .to_pandas()
#    .stack()
#    .stack()
#    .reset_index()
#    .assign(gridcell=gridcellnamingfunction, technology="Windonshore")
#    .set_index(["time", "technology", "gridcell"])
#    .drop(columns=["x", "y"])
#    .to_xarray()
# .round(
#    1
# )
# .loc[
#    :,
#    [0]
# ]
# )
# .set_index()
# .to_xarray()
# .loc[
#    :,
#    ["gridcell"]
# ]
# .to_csv(
#    "indices_region.csv",
#    header = False,
#    index = False
# )
# )  # .info()

## Offshore wind bottom


## Bottom mounted
### Areas


In [ ]:
geodata_files["offshore_bottom"]

In [ ]:
europe_offshore_bottom = (
    gpd.read_file(geodata_files["offshore_bottom"])
    .rename(columns={"id": "index"})
    .set_index("index")
    .rename({"United Kingdom": "UK", "GB": "UK"})
    # Since we do not need Svalbard and Jan Mayen, we drop them here. Since not all GeoDataFrames we are looping through
    # might have them, we ignore errors.
    .drop(["NO0B1", "NO0B2"], errors="ignore")
    # Here we filter to only include Norway.
    .filter(like=desired_country, axis=0)
)
europe_offshore_bottom

In [ ]:
europe_offshore_bottom.plot()

In [ ]:
year

In [ ]:
rectx1

In [ ]:
rectx2

In [ ]:
recty1

In [ ]:
recty2

In [ ]:
polygon = Polygon(
    [
        (rectx1, recty1),
        (rectx1, recty2),
        (rectx2, recty2),
        (rectx2, recty1),
        (rectx1, recty1),
    ]
)
europe_offshore_bottom = gpd.clip(europe_offshore_bottom, polygon)

In [ ]:
europe_offshore_bottom.plot(figsize=(15, 15))

In [ ]:
excluder_wind_offshore_bottom = ExclusionContainer()

In [ ]:
excluder_wind_offshore_bottom

In [ ]:
WDPA_Ia

In [ ]:
WDPA_Ib

In [ ]:
WDPA_II

In [ ]:
WDPA_III

In [ ]:
WDPA_IV

In [ ]:
excluder_wind_offshore_bottom.add_raster(WDPA_Ia)
excluder_wind_offshore_bottom.add_raster(WDPA_Ib)
excluder_wind_offshore_bottom.add_raster(WDPA_II)
excluder_wind_offshore_bottom.add_raster(WDPA_III)
excluder_wind_offshore_bottom.add_raster(WDPA_IV)

In [ ]:
#if snakemake.wildcards.spatial == "region" and snakemake.params.cutoffs["offwind"] != 0:
if snakemake.wildcards.spatial == "region" and float(snakemake.wildcards.cutoffs_offwind) != 0:
    low_cf = snakemake.output.cf_exclusion_windoff
    cutoff_raster("offwind", low_cf)
    excluder_wind_offshore_bottom.add_raster(low_cf)

In [ ]:
availability_matrix_wind_offshore_bottom = cutout.availabilitymatrix(
    europe_offshore_bottom, excluder_wind_offshore_bottom, nprocesses=snakemake.threads
)
availability_matrix_wind_offshore_bottom

In [ ]:
cap_per_sqkm = 1  # see if the results are the same when changing this

area = cutout.grid.set_index(["x", "y"]).to_crs(3035).area / 1e6

area = xr.DataArray(area, dims=("spatial"))

capacity_matrix_wind_offshore_bottom = (
    availability_matrix_wind_offshore_bottom.stack(spatial=["x", "y"])
    * area
    * cap_per_sqkm
)

# the following is the crucial addition
capacity_matrix_wind_offshore_bottom = capacity_matrix_wind_offshore_bottom.reindex(
    spatial=area.indexes.get("spatial")
)

capacity_matrix_wind_offshore_bottom

In [ ]:
if snakemake.wildcards.spatial == "grid":
    highRESareasWindOffshoreBottom = (
        capacity_matrix_wind_offshore_bottom.unstack()
        .stack(spatial=["index", "x", "y"])
        .to_pandas()
        .reset_index()
        .replace({"index": nuts_to_ssb})
        .assign(
            gridcell=gridcellnamingfunction,
            new_idx=lambda x: "Windoffshore." + x["index"] + "." + x["gridcell"],
        )
        .set_index("new_idx")
        .drop(columns=["index"])
        .loc[:, [0]]
        ## the following will drop rows with a value of zero, which will significantly reduce input file size.
        ## downside is that the indices need to be processed seperately then because gams needs an exhaustive list of all index values
        .rename(columns={0: "area"})
        .query("area != 0")
    )

if snakemake.wildcards.spatial == "region":
    highRESareasWindOffshoreBottom = (
        capacity_matrix_wind_offshore_bottom.sum("spatial")
        .to_pandas()
        .reset_index()
        .replace({"index": nuts_to_ssb})
        .assign(
            new_idx=lambda x: "Windoffshore." + x["index"] + "." + x["index"],
        )
        .set_index("new_idx")
        .drop(columns=["index"])
        .loc[:, [0]]
        .rename(columns={0: "area"})
        .query("area != 0")
    )

In [ ]:
highRESareasWindOffshoreBottom

In [ ]:
highRESareasWindOffshoreBottom.round(1).to_csv(
    snakemake.output["areaswindoffshore"], header=False, sep=" "
)

In [ ]:
offshore_bottom_turbine

In [ ]:
if snakemake.wildcards.spatial == "grid":
    _, capacity_factor_wind_offshore_bottom = cutout.wind(
        turbine=offshore_bottom_turbine, capacity_factor=True
    )

if snakemake.wildcards.spatial == "region":
    capacity_factor_wind_offshore_bottom = cutout.wind(
        turbine=offshore_bottom_turbine,
        matrix=capacity_matrix_wind_offshore_bottom,
        per_unit=True,
        # matrix=capacity_matrix_solar,
        capacity_factor=True,  # does not do anything, if you have no matrix or layout or shapes
        # per_unit=True,
        index=capacity_matrix_wind_offshore_bottom.index,
    )

In [ ]:
capacity_factor_wind_offshore_bottom

In [ ]:
# (
#    capacity_factor_wind_offshore_bottom
#    .stack(spatial=["x","y"])
#    .to_pandas()
#    .stack()
#    .stack()
#    .reset_index()
#    .assign(
#        gridcell = gridcellnamingfunction,
#        technology = "Windoffshorebottom"
#    )
#    .set_index(
#        ["time","technology","gridcell"]
#    )
#    .drop(
#        columns = ["x","y"]
#    )
#    .to_xarray()
#    #.round(
#    #    1
#    #)
#    #.loc[
#    #    :,
#    #    [0]
#    #]
#    #)
#    #.set_index()
#    #.to_xarray()
#    #.loc[
#    #    :,
#    #    ["gridcell"]
#    #]
#    #.to_csv(
#    #    "indices_region.csv",
#    #    header = False,
#    #    index = False
#    #)
# )#.info()

In [ ]:
##FLoating

In [ ]:
###Areas

In [ ]:
geodata_files["offshore_floating"]

In [ ]:
europe_offshore_floating = (
    gpd.read_file(geodata_files["offshore_floating"])
    .rename(columns={"id": "index"})
    .set_index("index")
    .rename({"United Kingdom": "UK", "GB": "UK"})
    # Since we do not need Svalbard and Jan Mayen, we drop them here. Since not all GeoDataFrames we are looping through
    # might have them, we ignore errors.
    .drop(["NO0B1", "NO0B2"], errors="ignore")
    # Here we filter to only include Norway.
    .filter(like=desired_country, axis=0)
)
europe_offshore_floating

In [ ]:
europe_offshore_floating.plot()

In [ ]:
year

In [ ]:
rectx1

In [ ]:
rectx2

In [ ]:
recty1

In [ ]:
recty2

In [ ]:
polygon = Polygon(
    [
        (rectx1, recty1),
        (rectx1, recty2),
        (rectx2, recty2),
        (rectx2, recty1),
        (rectx1, recty1),
    ]
)
europe_offshore_floating = gpd.clip(europe_offshore_floating, polygon)

In [ ]:
europe_offshore_floating.plot(figsize=(15, 15))

In [ ]:
excluder_wind_offshore_floating = ExclusionContainer()

In [ ]:
excluder_wind_offshore_floating

In [ ]:
WDPA_Ia

In [ ]:
WDPA_Ib

In [ ]:
WDPA_II

In [ ]:
WDPA_III

In [ ]:
WDPA_IV

In [ ]:
excluder_wind_offshore_floating.add_raster(WDPA_Ia)
excluder_wind_offshore_floating.add_raster(WDPA_Ib)
excluder_wind_offshore_floating.add_raster(WDPA_II)
excluder_wind_offshore_floating.add_raster(WDPA_III)
excluder_wind_offshore_floating.add_raster(WDPA_IV)

In [ ]:
if snakemake.wildcards.spatial == "region" and snakemake.params.cutoffs["offwind"] != 0:
    low_cf = snakemake.output.cf_exclusion_windoff_floating
    cutoff_raster("offwind_floating", low_cf)
    excluder_wind_offshore_floating.add_raster(low_cf)

In [ ]:
availability_matrix_wind_offshore_floating = cutout.availabilitymatrix(
    europe_offshore_floating, excluder_wind_offshore_floating, nprocesses=snakemake.threads
)
availability_matrix_wind_offshore_floating

In [ ]:
ap_per_sqkm = 1  # see if the results are the same when changing this

area = cutout.grid.set_index(["x", "y"]).to_crs(3035).area / 1e6

area = xr.DataArray(area, dims=("spatial"))

capacity_matrix_wind_offshore_floating = (
    availability_matrix_wind_offshore_floating.stack(spatial=["x", "y"])
    * area
    * cap_per_sqkm
)

# the following is the crucial addition
capacity_matrix_wind_offshore_floating = capacity_matrix_wind_offshore_floating.reindex(
    spatial=area.indexes.get("spatial")
)

capacity_matrix_wind_offshore_floating

In [ ]:
if snakemake.wildcards.spatial == "grid":
    highRESareasWindOffshoreFloating = (
        capacity_matrix_wind_offshore_floating.unstack()
        .stack(spatial=["index", "x", "y"])
        .to_pandas()
        .reset_index()
        .replace({"index": nuts_to_ssb})
        .assign(
            gridcell=gridcellnamingfunction,
            new_idx=lambda x: "Windoffshorefloating." + x["index"] + "." + x["gridcell"],
        )
        .set_index("new_idx")
        .drop(columns=["index"])
        .loc[:, [0]]
        ## the following will drop rows with a value of zero, which will significantly reduce input file size.
        ## downside is that the indices need to be processed seperately then because gams needs an exhaustive list of all index values
        .rename(columns={0: "area"})
        .query("area != 0")
    )

if snakemake.wildcards.spatial == "region":
    highRESareasWindOffshoreFloating = (
        capacity_matrix_wind_offshore_floating.sum("spatial")
        .to_pandas()
        .reset_index()
        .replace({"index": nuts_to_ssb})
        .assign(
            new_idx=lambda x: "Windoffshorefloating." + x["index"] + "." + x["index"],
        )
        .set_index("new_idx")
        .drop(columns=["index"])
        .loc[:, [0]]
        .rename(columns={0: "area"})
        .query("area != 0")
    )


In [ ]:
highRESareasWindOffshoreFloating

In [ ]:


highRESareasWindOffshoreFloating.round(1).to_csv(
    snakemake.output["areaswindoffshorefloating"], header=False, sep=" "
)

In [ ]:
### Capacity factors

In [ ]:
offshore_floating_turbine

In [ ]:
if snakemake.wildcards.spatial == "grid":
    _, capacity_factor_wind_offshore_floating = cutout.wind(
        turbine=offshore_floating_turbine, capacity_factor=True
    )

if snakemake.wildcards.spatial == "region":
    capacity_factor_wind_offshore_floating = cutout.wind(
        turbine=offshore_floating_turbine,
        matrix=capacity_matrix_wind_offshore_floating,
        per_unit=True,
        # matrix=capacity_matrix_solar,
        capacity_factor=True,  # does not do anything, if you have no matrix or layout or shapes
        # per_unit=True,
        index=capacity_matrix_wind_offshore_floating.index,
    )

In [ ]:
capacity_factor_wind_offshore_floating

# highRES-export-capacity_factors


Now we need to get it in the correct output format:


In [ ]:
capacity_factor_solar.name = None

In [ ]:
# capacity_factor_wind_onshore.name = None

In [ ]:
capacity_factor_solar.name = "Solar"
# capacity_factor_wind_onshore.name = "Windonshore"
capacity_factor_wind_offshore_bottom.name = "Windoffshore"
capacity_factor_wind_offshore_floating.name = "Windoffshorefloating"

https://stackoverflow.com/questions/56457280/merging-xarray-files-into-a-new-dimension


In [ ]:
concatlist = [
    capacity_factor_solar,
    # capacity_factor_wind_onshore,
    capacity_factor_wind_offshore_bottom,
    capacity_factor_wind_offshore_floating,
]

In [ ]:
for i in concatlist:
    print(i.name)

In [ ]:
range(0, len(capacity_factor_solar.time))

In [ ]:
xr.concat(concatlist, pd.Index(list(da.name for da in concatlist), name="technology"))

In [ ]:
gridcellnamingfunctionxr = (
    lambda da: "x"
    + (da.x * 100).astype("int").astype("str").astype("object")
    + "y"
    + (da.y * 100).astype("int").astype("str").astype("object")
)

In [ ]:
year

In [ ]:
if snakemake.wildcards.spatial == "grid":
    (
        xr.concat(
            concatlist, pd.Index(list(da.name for da in concatlist), name="technology")
        )
        .stack(spatial=["x", "y"])
        .assign_coords(gridcell=gridcellnamingfunctionxr)
        .set_index(spatial="gridcell")
        .drop_vars(["lat", "lon"])
        .assign_coords(time=range(0, len(concatlist[0].time)))
        .stack(final=["time", "technology", "spatial"])
        .round(2)
        .to_pandas()
        .to_csv(snakemake.output["capfacfile"])
    )

if snakemake.wildcards.spatial == "region":
    (
        xr.concat(
            concatlist, pd.Index(list(da.name for da in concatlist), name="technology")
        )
        # .stack(spatial=["x", "y"])
        # .assign_coords(gridcell=gridcellnamingfunctionxr)
        # .set_index(spatial="gridcell")
        # .drop_vars(["lat", "lon"])
        .assign_coords(time=range(0, len(concatlist[0].time)))
        .rename({"index": "spatial"})
        .stack(final=["time", "technology", "spatial"])
        .round(2)
        .to_pandas()
        .to_frame()
        .reset_index()
        .replace({"spatial": nuts_to_ssb})
        .fillna(0)
        .set_index(["time", "technology", "spatial"])
        .to_csv(snakemake.output["capfacfile"])
    )

Next, create the GDX file:
`/opt/gams/gams36.2_linux_x64_64_sfx/csv2gdx vre_1997.csv output=vre_1997.gdx ID=vre_gen Index='(1,2,3)' Value='(4)' UseHeader=True StoreZero=True`

To have hydro:

`/opt/gams/gams36.2_linux_x64_64_sfx/gdxdump vre_2013_dev.gdx output=vre_2013_dev.csv symb=vre_gen format=csv`

`grep ',"HydroRoR","PT",' vre_2013_dev.csv > vre_2013_dev_hydro_PT.csv`

search and replace "

`cat vre_pt2013.csv ../model/data/vre_2013_dev_hydro_PT.csv | sort -g > out.csv`
